# Blackstrap Service

The Blackstrap service proxies a local file service for situations where there may be a sandbox and it is desirable to use a hostname in the url.

Normally the file url i

In [10]:
from fs.osfs import OSFS
from fs.errors import CreateFailed
from fs.info import Info
from socket import gethostname
from urllib.parse import urlparse
import six
from os import statvfs



"""
0.3
changed: initClass() --> initHost(name)
changed: addShare() removed hostname
changed: __init__() use url to init
changed: subclasses OSFS

0.2c
added: generateShareId(hostname, sharename)
changed: shareId is now 'share.host' subdomain style
         to be able to map to a file name.
fixed: now calls super class init
fixed: missing import SubFS
fixed: now passthru all fs methods
"""

def generateShareId(hostname, sharename):
    return sharename +'.'+ hostname

class BlackstrapFS(OSFS):
    
    
    ###### class methods ######

    # class method
    def initHost(hostname=None):
        """
        0.1
        The BlackstrapFS serves files from
        a virtual context. The context is
        kept inside the class.
        """
        if hostname is None:
            BlackstrapFS.__hostname = gethostname()
        else:
            BlackstrapFS.__hostname = hostname
        BlackstrapFS.__shares = dict()
        
        return BlackstrapFS
    
    
    
    # class method
    def addShare(srcaddr,
                 sharename
                ):
        """
        0.1
        Shares are added to the class itself.
        An instance of the service can
        select which share to serve.
        """
        
        #todo: check for blank hostname
        #todo: check for existing before...
        
        shares = BlackstrapFS.__shares
        
        shareid = generateShareId( 
            BlackstrapFS.__hostname, 
            sharename)
        
        shares[shareid] = srcaddr
        
        return BlackstrapFS
    
    # class method
    def closeHost():
        pass
    
    ###### Instance methods ######
    
    def __init__(self,urlstr,*args):
        # todo: require file scheme
        self._urlstr = urlstr
        
        url = urlparse(urlstr)
        self.url = url
        shares = BlackstrapFS.__shares
        shareid = url.netloc
        
        try:
            sharepath = shares[shareid]
            realpath = sharepath+'/'+url.path
        except:
            raise CreateFailed(shareid)
        
        self.__addr = realpath
        super().__init__(realpath,*args)
        
    def getinfo(self, path='/',namespaces=None):
        info_raw = super().getinfo(path, namespaces).raw
        if namespaces != None and 'limits' in namespaces:
            st = statvfs(self.__addr)
            if st != None:
                bsize = st.f_bsize
                total = bsize * st.f_blocks
                used = total - (bsize * st.f_bfree)
                # free = bsize * st.f_bavail
                free = total - used
                info_raw['limits'] = {
                    self._urlstr: {
                        'total': total,
                        'used': used,
                        'free': free                        
                    }
                }
        return Info(info_raw)
        
        
    def geturl(self,path):
        # if path starts with '/'
        _urlstr = self._urlstr.rstrip('/')
        _path = path.lstrip('/')
        if(_path == ""):
            # if there is no path,
            # we must return _urlstr as inited.
            # It may match an entry in the fstab.
            return self._urlstr
        return f"{_urlstr}/{_path}"
        
        
    def _close(self):
        pass
    
    if six.PY2:
        def close(self):  # noqa: D102
            self._close()
            super(BlackstrapFS, self).close()
    else:
        def close(self): # noqa: D102
            self._close()
            super().close()
                    
            
    ####################
    #  cosmos API      #
    ####################
    
    # DEPRICATE THIS
    # DO NOT ADD EXTERNAL API

    def lookup(self, path, base=None):
        # FIXME: Placeholder.
        return True
    

In [11]:
hostname = gethostname()

BlackstrapFS.initHost('localhost')
BlackstrapFS.addShare(
    srcaddr = 'fs',
    sharename = 'fs'
).addShare(
    srcaddr = 'fs2',
    sharename = 'fs2'
)

fs1 = BlackstrapFS('file://fs.localhost/')
fs1.getinfo('/').raw


{'basic': {'name': '', 'is_dir': True}}

In [12]:
fs1.getinfo('/', namespaces=['limits']).raw

{'basic': {'name': '', 'is_dir': True},
 'limits': {'file://fs.localhost/': {'total': 121938661376,
   'used': 99794636800,
   'free': 22144024576}}}

In [9]:
from fs.opener import open
from socket import gethostname

print(gethostname())
open('file://fs')

mbp-linux


(OSFS('/home/raygan/Cosms/Dboy/Laydbug/dev/cburnfs_py/cburnfs/fs'), None)